In [45]:

import copy

import sys
sys.path.append("/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/recnn/")

import argparse
import os
from subprocess import check_call
import sys
import numpy as np
import utils
import time
import logging
import torch
import torch.optim as optim
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
import pickle
import model.data_loader as dl
import model.dataset as dataset
from model import recNet as net
from model import preprocess 

In [48]:
test0 = pickle.load(open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/raw_trees/ginkgo_jets0.pkl',"rb"))
test1 = pickle.load(open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/raw_trees/ginkgo_jets1.pkl',"rb"))
test2 = pickle.load(open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/raw_trees/ginkgo_jets2.pkl',"rb"))


antikt_jets = pickle.load(open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/reclustered_trees/ginkgo_antikt.pkl',"rb"))
truth_jets = pickle.load(open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/reclustered_trees/ginkgo_truth.pkl',"rb"))
ptasc_jets = pickle.load(open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/reclustered_trees/ginkgo_ptasc.pkl',"rb"))

ptdesc_jets = pickle.load(open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/reclustered_trees/ginkgo_ptdesc.pkl',"rb"))

In [49]:
print(len(test0)+len(test1)+len(test2))
print(len(antikt_jets),len(ptdesc_jets),len(ptasc_jets),len(truth_jets))

48
48 48 48 48


In [3]:
### Get the 4-vector pT
def _pt(v):
    pt=(v[0:2] ** 2).sum() ** 0.5
    return pt

In [4]:
def permute_by_pt(jet, root_id=None):
    # ensure that the left sub-jet has always a larger pt than the right

    if root_id is None:
        root_id = jet["root_id"]

    if jet["tree"][root_id][0] != -1: # because if it is -1, then there are no children and so it is a leaf (SM)
        left = jet["tree"][root_id][0] #root_id left child. This gives the id of the left child (SM)
        right = jet["tree"][root_id][1] #root_id right child. This gives the id of the right child (SM)
        #So object["tree"][root_id] contains the position of the left and right children of object in jet["content"] (SM)

        pt_left = _pt(jet["content"][left]) #We get the pt of the left child (SM)
        pt_right = _pt(jet["content"][right]) #We get the pt of the right child (SM)
#         print('pt_left=',pt_left)
#         print('pt_right=',pt_right)

        if pt_left < pt_right: # We switch the left and right children if necessary so that the left child has greater pt (SM) 
            jet["tree"][root_id][0] = right
            jet["tree"][root_id][1] = left
            print('pt_left=',pt_left)
            print('pt_right=',pt_right)

        # We call the function recursively to ensure each subtree satisfies this property (SM)    
        permute_by_pt(jet, left) 
        permute_by_pt(jet, right)

    return jet


# We do the same as in the previous function but sorting by pT in this case (SM) 
def sequentialize_by_pt(jet, reverse=False):
    # transform the tree into a sequence ordered by pt

    jet = copy.deepcopy(jet)

#     print('jet["tree"]=',jet["tree"])
#     print('---'*20)

    leaves = np.where(jet["tree"][:, 0] == -1)[0] #The entries give the position of the leaves in the content list 
#     print('leaves=',leaves)
#     print('---'*20)
    
    nodes = [n for n in leaves]# Same as leaves
#     print('nodes=',nodes)
#     print('---'*20)    
    
    content = [jet["content"][n] for n in nodes] #Here we get the leaves content
#     print('content=',content)
#     print('---'*20)  
        
    nodes = [i for i in range(len(nodes))]
    tree = [[-1, -1] for n in nodes] #We make a tree of length=Number of leaves, where the entries are all (-1), as expected for leaves
#     print('tree=',tree)
#     print('---'*20)     
    
    #Order the list of leaves based on pT. reverse=False gives the list in ascending pT
    pool = sorted([n for n in nodes],
                  key=lambda n: _pt(content[n]),
                  reverse=reverse)
#     print('pool=',pool)
#     print('Ordered pT content=',[ _pt(content[n]) for n in pool])
#     print('---'*20)                  
                  
    next_id = len(pool)

    #Remake the tree. We start from the end of the ordered list of leaves. So in reverse=False we cluster first the greatest pT constituents. => reverse=False gives a tree ordered in descending order in pT
    while len(pool) >= 2:
        right = pool[-1]
        left = pool[-2]
#         print('right=',_pt(content[right]))
#         print('left=',_pt(content[left]))
#         print('---'*20)       
        del pool[-1]
        del pool[-1]

        # We build the tree as a ladder. So we append the right subjet to the list 
        nodes.append(next_id)
#         print('next_id=',next_id)
#         print('---'*20)        
        c = (content[left] + content[right])

        if len(c) == 5:
            c[-1] = -1

        content.append(c) # We append the new subjet to the contents list
#         print('pT c=',_pt(c))
#         print('pT content[next_id]=c?',_pt(content[next_id]))
        tree.append([left, right]) # We append the children locations of the new subjet (locations in the contents list)
#         print('tree=',tree)
#         print('tree[next_id]=',tree[next_id])
#         print('---'*20)
#         print('---'*20)
        pool.append(next_id) #We append the location of the new subjet 'c', so content[next_id]=c
        next_id += 1

    jet["content"] = np.array(content)
    jet["tree"] = np.array(tree).astype(int)
    jet["root_id"] = len(jet["tree"]) - 1

    return jet


In [19]:
sig = [ptdesc_jets[k][0] for k in range(len(ptdesc_jets))if ptdesc_jets[k][1]==1]
bg = [ptdesc_jets[k][0] for k in range(len(ptdesc_jets))if ptdesc_jets[k][1]==0]
print(len(antikt_jets),len(sig),len(bg))
ptdesc_jets

8 8 0


[({'root_id': 4,
   'tree': array([[-1, -1],
          [-1, -1],
          [-1, -1],
          [ 2,  0],
          [ 1,  3]]),
   'content': array([[ 56.14388801,  30.86706416,  32.48531297,  33.77548634],
          [438.49541483, 254.35404778, 254.63201057, 250.45634994],
          [ 57.5381818 ,  33.47622328,  31.58001175,  34.46549908],
          [113.68206981,  64.34328743,  64.06532472,  68.24098541],
          [552.17748464, 318.69733522, 318.69733529, 318.69733535]]),
   'LambdaRoot': tensor(3.0817),
   'Lambda': tensor(3.0817),
   'Delta_0': tensor(196.),
   'pt_cut': 36.0,
   'algorithm': 'truth',
   'deltas': array([196.        ,   0.        , 105.28229523,   0.        ,
            0.        ]),
   'draws': array([None, tensor(0.2347), tensor(0.5372), tensor(0.2521),
          tensor(0.0458)], dtype=object),
   'leaves': array([[ 56.14388801,  30.86706416,  32.48531297,  33.77548634],
          [438.49541483, 254.35404778, 254.63201057, 250.45634994],
          [ 57.5381818 

In [5]:
from model import preprocess 

test = antikt_jets[0][1][0]
rewritten = preprocess.rewrite_content(test)
permute = preprocess.permute_by_pt(test)
ptdesc = preprocess.sequentialize_by_pt(test,reverse=True)
ptasc = preprocess.sequentialize_by_pt(test,reverse=False)
# print(test['content']-rewritten['content'],test['tree']-permute['tree'],test['content']-permute['content'])
# print(ptdesc['tree'],test['tree'])
# print(ptasc['tree'],ptdesc['tree'])
print(test.keys())
#print(preprocess.extract_nyu_samples(ptdesc)['label'],ptdesc['label'])

TypeError: 'int' object is not subscriptable

In [38]:
#### Change the topology of the tree first, if necessary 
algo = 'kt'
if algo=='ptdesc':
    sequentialize_by_pt(Reverse=True)
if algo=='ptasc':
    sequentialize_by_pt(Reverse=False)
else:
    permute_by_pt
toptag_reference_train_x = [preprocess.extract_nyu_samples(preprocess.permute_by_pt(preprocess.rewrite_content(jet))) for jet in toptag_reference_train_x]


#### Instead of dataloader, since jets are already in "right" form (like toptag): 

file_name = '/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/reclustered_trees/Clustering_antikttree_ginkgo_jets1.pkl'
with open(file_name, "rb") as f: 
    file_name =pickle.load(f,encoding='latin-1') 


antikt_x=np.asarray([x for (x,y) in file_name])
antikt_y=np.asarray([y for (x,y) in file_name])



In [80]:
import random
from sklearn.utils import check_random_state

train_frac_rel = .6
val_frac_rel = .2
test_frac_rel = .2

data_list = truth_jets
sig=[data_list[k][0] for k in range(len(data_list))if data_list[k][1]==1]
bkg=[data_list[k][0] for k in range(len(data_list))if data_list[k][1]==0] 

print(len(sig_list),len(bkg_list),len(data_list))

rndstate = random.getstate()
random.seed()
size=np.minimum(len(sig),len(bkg))
sig_label=np.ones((size),dtype=int)
bkg_label=np.zeros((size),dtype=int)

train_frac=train_frac_rel
val_frac=train_frac+val_frac_rel
test_frac=val_frac+test_frac_rel

N_train=int(train_frac*size)
Nval=int(val_frac*size)
Ntest=int(test_frac*size)

print(N_train,Nval,Ntest)

indices_sig = check_random_state(None).permutation(len(sig))

sig = [sig[indices_sig[i]] for i in range(len(sig))]


28 20 48
12 16 20


In [81]:
def split_shuffle_sample(sig, bkg, train_frac_rel, val_frac_rel, test_frac_rel):
    '''
    Function to split the sample into train, cross-validation and test with equal number of sg and bg events. Then shuffle each set.
    Args:
      sig: signal sample
      bkg: background sample
      train_frac_rel: fraction of data for the train set
      val_frac_rel: fraction of data for the validation set
      test_frac_rel: fraction of data for the test set
    '''
    print('---'*20)
    print('Loading and shuffling the trees ...')
  
    rndstate = random.getstate()
    random.seed()
    size=np.minimum(len(sig),len(bkg))
#     print('sg length=',len(sig))
  
    sig_label=np.ones((size),dtype=int)
    bkg_label=np.zeros((size),dtype=int)
    ##-----------------------------------------------
    print('Creating train, val and test datasets ...')
    
    # Split data into train, val and test
    train_frac=train_frac_rel
    val_frac=train_frac+val_frac_rel
    test_frac=val_frac+test_frac_rel

    N_train=int(train_frac*size)
    Nval=int(val_frac*size)
    Ntest=int(test_frac*size)
    
#     print('len(sg)=',len(sig))
#     print('len(bkg)=',len(bkg))
#     print('size=',size)
#     print('N_train=',N_train)
#     print('Nval=',Nval)
#     print('Ntest=',Ntest)
#     print('+-+-'*20)


    # Shuffle sig and bkg sets independently
    indices_sig = check_random_state().permutation(len(sig))
    sig = sig[indices_sig]

    indices_bkg = check_random_state(None).permutation(len(bkg))
    bkg = [bkg[indices_bkg[i]] for i in range(len(bkg))]



    ##-----------------------------------------------
    # Concatenate sg and bg data
    train_x=np.concatenate((sig[0:N_train],bkg[0:N_train]))
    train_y=np.concatenate((sig_label[0:N_train],bkg_label[0:N_train]))
  
    dev_x=np.concatenate((sig[N_train:Nval],bkg[N_train:Nval]))
    dev_y=np.concatenate((sig_label[N_train:Nval],bkg_label[N_train:Nval]))
  
    test_x=np.concatenate((sig[Nval:Ntest],bkg[Nval:Ntest]))
    test_y=np.concatenate((sig_label[Nval:Ntest],bkg_label[Nval:Ntest]))

#     print('train_x=',[x['charge']for x in train_x] )
#     print('dev_x=',[x['charge']for x in dev_x])
#     print('test_x=',[x['charge']for x in test_x])
    ##-----------------------------------------------
    # Shuffle the sets
    indices_train = check_random_state(None).permutation(len(train_x))
    print('train_y=',train_y)
    print('train x shape=',train_x.shape)
    print('///'*20)
    train_x = [train_x[indices_train[i]] for i in range(len(train_x))]
    train_y = [train_y[indices_train[i]] for i in range(len(train_y))]
    print('train_y=',train_y)
  
    indices_dev = check_random_state(None).permutation(len(dev_x))
    dev_x = [dev_x[indices_dev[i]] for i in range(len(dev_x))]
    dev_y = [dev_y[indices_dev[i]] for i in range(len(dev_y))]
  
    indices_test = check_random_state(None).permutation(len(test_x))
    test_x = [test_x[indices_test[i]] for i in range(len(indices_test))]
    test_y = [test_y[indices_test[i]] for i in range(len(test_y))]
    
#     print('train_x=',[x['charge']for x in train_x] )
#     print('dev_x=',[x['charge']for x in dev_x])
#     print('test_x=',[x['charge']for x in test_x])

    
    ##----------------------------------------------- 
    train_x=np.asarray(train_x)
    dev_x=np.asarray(dev_x)
    test_x=np.asarray(test_x)
    train_y=np.asarray(train_y)
    dev_y=np.asarray(dev_y)
    test_y=np.asarray(test_y)
    
#     Uncomment below if we change the NN output to be of dim=1
#     train_y=np.asarray(train_y).reshape((-1,1))
#     dev_y=np.asarray(dev_y).reshape((-1,1))
#     test_y=np.asarray(test_y).reshape((-1,1))
  
    print('Train shape=',train_x.shape)
    print('Val shape=',dev_x.shape)
    print('Test shape=',test_x.shape)
#     print('train=',train_x[0]['content'])
#     print('test_y=',test_y)
   
    return train_x, train_y, dev_x, dev_y, test_x, test_y


In [86]:
in_filename = 'ginkgo_test.pkl'
str(in_filename).split('.')[0]

'ginkgo_test'